# Deploy Fraud Model using [Managed Online Endpoints](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-managed-online-endpoints)

Now we'll deploy the model as a Online Endpoint in the Azure ML Workspace. 

Managed online endpoints help to deploy your ML models in a turnkey manner. Managed online endpoints work with powerful CPU and GPU machines in Azure in a scalable, fully managed way. Managed online endpoints take care of serving, scaling, securing, and monitoring your models, freeing you from the overhead of setting up and managing the underlying infrastructure. The main example in this doc uses managed online endpoints for deployment.

## Install dependencies
We need to remove the old machine learning extension and replace it for the [ml v2 extension](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-cli).

In [ ]:
!az extension remove -n azure-cli-ml 

In [ ]:
!az extension add -n ml -y

### Login on Azure and Configure Subscription and AML Workspace

In [ ]:
!az login --use-device-code

In [ ]:
!az account set --subscription <your-subscription>
!az configure --defaults workspace=<your-aml-workspace> group=<your-resource-group>

## Deploy the Model

In [ ]:
!az ml online-endpoint create --name <ENDPOINT-NAME> -f ../deployment/endpoint.yml

In [ ]:
!az ml online-deployment create --name fraud-ga --endpoint <ENDPOINT-NAME> -f ../deployment/deployment.yml --all-traffic

#### Call the API

In [27]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {"type":3.0,
        "amount":1864.28,
        "oldbalanceOrg":21249.0,
        "newbalanceOrig": 19384.72,
        "oldbalanceDest":0.0,
        "newbalanceDest":0.0,
        "hour":1.0,
        "dayOfMonth":1.0,
        "isMerchantDest":1.0,
        "errorBalanceOrig":0.0,
        "errorBalanceDest":1864.28}

body = str.encode(json.dumps(data))

url = '<YOUR-ENDPOINT>'
api_key = '<YOUR-ENDPOINT-KEY>' # Replace this with the API key for the web service

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'fraud-ga' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'{"prediction": "0"}'
